## Step1 导入相关包

In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import AutoTokenizer,AutoModelForSequenceClassification

## Step2 加载数据

In [3]:
import pandas as pd
data = pd.read_csv('ChnSentiCorp_htl_all.csv')
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [5]:
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


## Step3 创建Dataset

In [6]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv('./ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna()

    def __getitem__(self, index):
        data = self.data.iloc[index]
        return data['review'], data['label']

    def __len__(self):
        return len(self.data)

In [7]:
dataset=MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [8]:
from torch.utils.data import random_split

trainset,validset = random_split(dataset,lengths=[0.9,0.1])
len(trainset),len(validset)

(6989, 776)

In [9]:
for i in range(10):
    print(trainset[i])

('每次来盐城都是住的这里，感觉在盐城虽算不上最好的饭店，但还是算不错的饭店了。房间的环境稍显有些破旧，特别是地毯有点脏。但是整体环境及性价比，感觉还是不错的。补充点评2008年3月22日：服务态度也还不错，后来我一朋友凌晨三点过来的，小姐主动提出，如果他12点前退房，只收他半天房钱。', 1)
('酒店房间设施陈旧，尤其卫生间设置不合理，洗澡很不方便。前台服务人员态度极其冷淡。', 0)
('的确很靠近火车站及汽车北站，走路不超过15分钟。房间里摆放了3本推荐的新书，其中一本是童话双语的，感觉很好。不知道为什么房间里不备电话簿。', 1)
('景观不错,服务也不错,早餐也不错,地段不错,去哪都方便', 1)
('此次入住感觉不错,销售部服务到位,大堂经理能理解客人的需要,做出相应的措施,下次还要入住....', 1)
('大堂较小，房间不错，离地铁四号线非常近，交通比较方便。', 1)
('成都完全不象是有过地震的样子，满街火锅飘香，北边一点是一个公园，到六七点都是散步、溜狗的，很是热闹。酒店只是楼层低了点，所以临街的房间早上会被车吵醒。酒店的餐厅很好，偏贵，过马路有一个川菜馆，很好，还有n多个火锅店，可惜一个人没法吃。跑题了：房间很好，checkin/out都很快，早餐不错。打车困难一点，不知道是为什么。宾馆反馈2008年7月17日：感谢您在震后与我们一起感受成都的震后生活,同时感谢您对我们提出宝贵的意见,在上下班高峰期我们会遇见打车难的问题,建议您在以后出门前先致电我们礼宾处,我们礼宾将提前安排好用车。', 1)
('酒店装修气味很浓，房间感觉一般，与4星有不少差距。入住期间居然不开空调，热得要死，刚好房间又向阳，期间打了几次电话，最后同价升级成单人套房，郁闷的心情总算好点！卫浴品太旧给人感觉不爽，总之感觉不好，下次不会再住！！', 0)
('房间太小，服务不错。在王府井能这样就不错了', 1)
('3月19日入住,共两晚.现在才上来做下点评,呵呵.1,定的是双人床大房,虽然是比较年头多的宾馆,但是整洁程度和环境还算不错了!2,服务人员比较热情,由于自己的充电器宾馆内使用不了.所以半夜的时候要求前台送个旅行充电器过来.但是前三个都不好用,服务生上上下下的跑了四五次!但没有一点厌烦!最后还一再向我表示歉意!很棒!!!3,餐饮方面西餐厅感觉还不错!价格和味道方面还

## Step5 创建DataLoader

In [10]:
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts,labels = [],[]
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])

    inputs = tokenizer(texts, max_length=128,padding='max_length',truncation=True,return_tensors='pt')
    inputs['labels'] = torch.tensor(labels)
    return inputs


In [ ]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=collate_func)
valiloader = DataLoader(validset,batch_size=64,shuffle=False,collate_fn=collate_func)

In [17]:
next(enumerate(valiloader))

(0,
 {'input_ids': tensor([[ 101, 6821, 3221,  ...,    0,    0,    0],
         [ 101, 2139, 3208,  ...,    0,    0,    0],
         [ 101,  769, 6858,  ..., 6392, 6369,  102],
         ...,
         [ 101, 4289, 3300,  ...,    0,    0,    0],
         [ 101, 4384, 1862,  ...,    0,    0,    0],
         [ 101, 4384, 1862,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 

## Step6 创建模型及优化器

In [19]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model.parameters()

<generator object Module.parameters at 0x7fa3f8feb740>

In [20]:
optimizer = Adam(model.parameters(),lr=2e-5)

## Step7 训练与验证

In [22]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits,dim=-1)
            acc_num += (pred.long() == batch['labels'].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3,log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step ==0:
                print(f"ep:{ep}, global_step:{global_step}, loss:{output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"e:{ep}, acc:{acc}")


## Step8 模型训练

In [23]:
train()

ep:0, global_step:0, loss:0.6272197365760803
ep:0, global_step:100, loss:0.34883877635002136
ep:0, global_step:200, loss:0.280170738697052
e:0, acc:8.19329833984375
ep:1, global_step:300, loss:0.2533099949359894
ep:1, global_step:400, loss:0.12583114206790924
e:1, acc:8.362112998962402
ep:2, global_step:500, loss:0.16282659769058228
ep:2, global_step:600, loss:0.030868077650666237
e:2, acc:8.60180377960205
